In [26]:
import random
import simpy
import itertools

In [186]:
MIN_PRIORITY = 0
MAX_PRIORITY = 100

class PowerPlant(object):
    def __init__(self, env, name, priority, min_production, max_production, initial_production):
        self.env = env
        self.name = name
        self.max_production = max_production
        self.min_production = min_production
        self.previous_production = initial_production
        self.next_min = min_production
        self.next_max = max_production
        if priority < MIN_PRIORITY:
            priority=MIN_PRIORITY
        elif priority > MAX_PRIORITY:
            priority=MAX_PRIORITY
        self.priority = priority

            
    def print_name(self):
        print(self.name)
        
    def produce(self, area, production):
        self.previous_production = production
        print('Power plant ' + self.name + ' produced ' + str(production) + ' of energy')  
        if production > 0:
            yield area.demand.get(production)
        
    def give_next_min_production(self):
        return self.next_min
    
    def give_next_max_production(self):
        return self.next_max

In [189]:
class ElectricityUser(object):
    def __init__(self, env, name, consumption):
        self.env = env
        self.name = name
        self.consumption = consumption
        
    def print_name(self):
        print(self.name)
        
    def consume(self,area):
        if self.consumption > 0:
            yield area.demand.put(self.consumption)

In [190]:
class Area(object):
    def __init__(self, env, name):
        self.env = env
        self.name = name
        self.users = []
        self.demand = simpy.Container(env)
        self.providers = []
    
        
    def add_electricity_user(self, name,cap):
        factory = ElectricityUser(env,name,cap)
        self.users.append(factory)
        
    def list_electricity_users(self):
        for user in self.users:
            print(user.name)

    def add_electricity_provider(self, name,  priority, min_production, max_production, initial_production):
        power_station = PowerPlant(env,name,  priority, min_production, max_production, initial_production)
        self.providers.append(power_station)
        
    def list_electricity_providers(self):
        for provider in self.providers:
            print(provider.name)
            
    def sort_electricity_providers(self):
        random.shuffle(self.providers)
        self.providers = sorted(self.providers, key=lambda provider: provider.priority)
            
    def show_demand(self):
        print(self.demand.level)

In [207]:
env = simpy.Environment()
alue = Area(env, 'alue')
alue.add_electricity_user('tehdas1',140)
alue.list_electricity_users()
alue.add_electricity_provider('priority4a',4,0,100,50)
alue.add_electricity_provider('priority1',1,0,100,50)
alue.add_electricity_provider('priority4b',4,0,100,50)
alue.list_electricity_providers()

tehdas1
priority4a
priority1
priority4b


In [208]:
def produce_in_area(env, area):
    area.sort_electricity_providers()
    minimum_production = 0
    for provider in area.providers:
        minimum_production += provider.give_next_min_production()
    difference = area.demand.level - minimum_production
    if difference <= 0:
        for provider in area.providers:
            yield env.process(user.produce(area,provider.give_next_min_production()))
    else:
        for provider in area.providers:
            next_max = provider.give_next_max_production()
            next_min = provider.give_next_min_production()
            delta = next_max-next_min
            if delta <= difference:
                yield env.process(provider.produce(area, next_max))
                difference -= delta
            else:
                yield env.process(provider.produce(area, next_min+difference))
                difference = 0




def time_step(env, area):
    print(area.name)
    for i in itertools.count():
        yield env.timeout(1)
        for user in area.users:
            user.print_name()
            yield env.process(user.consume(area))
        area.show_demand()
        yield env.process(produce_in_area(env,area))
                

In [209]:
env.process(time_step(env,alue))

<Process(time_step) object at 0x7f453cc63130>

In [210]:
env.run(until=7)

alue
tehdas1
140
Power plant priority1 produced 100 of energy
Power plant priority4a produced 40 of energy
Power plant priority4b produced 0 of energy
tehdas1
140
Power plant priority1 produced 100 of energy
Power plant priority4b produced 40 of energy
Power plant priority4a produced 0 of energy
tehdas1
140
Power plant priority1 produced 100 of energy
Power plant priority4a produced 40 of energy
Power plant priority4b produced 0 of energy
tehdas1
140
Power plant priority1 produced 100 of energy
Power plant priority4a produced 40 of energy
Power plant priority4b produced 0 of energy
tehdas1
140
Power plant priority1 produced 100 of energy
Power plant priority4a produced 40 of energy
Power plant priority4b produced 0 of energy
tehdas1
140
Power plant priority1 produced 100 of energy
Power plant priority4a produced 40 of energy
Power plant priority4b produced 0 of energy


In [178]:
alue.show_demand()

0


In [133]:
alue.list_electricity_users()

tehdas1


In [12]:
SIM_TIME = 20
RANDOM_SEED = 13
NUM_MACHINES = 2
WASHTIME = 2
T_INTER = 3

In [8]:
class Carwash(object):
    def __init__(self, env, num_machines, washtime):
        self.env = env
        self.machine = simpy.Resource(env, num_machines)
        self.washtime = washtime
        
    def wash(self, car):
        yield self.env.timeout(WASHTIME)
        print("Carwash removed %d%% of %s's dirt." %
              (random.randint(50,99), car))

In [9]:
def car(env, name, cw):
    print('%s arrives at the carwas at %.2f.' % (name, env.now))
    with cw.machine.request() as request:
        yield request
        
        print('%s enters the carwash at %.2f.' % (name, env.now))
        yield env.process(cw.wash(name))
        
        print('%s leaves the carwash at %.2f.' % (name, env.now))

In [10]:
def setup(env, num_machines, washtime, t_inter):
    carwash = Carwash(env, num_machines, washtime)
    
    for i in range(4):
        env.process(car(env, 'Car %d' % i, carwash))
        
    while True:
        yield env.timeout(random.randint(t_inter - 2, t_inter + 2))
        i += 1
        env.process(car(env, 'Car %d' % i, carwash))
        

In [11]:
print('Carwash')
random.seed(RANDOM_SEED)

env = simpy.Environment()
env.process(setup(env, NUM_MACHINES, WASHTIME, T_INTER))

env.run(until=SIM_TIME)

Carwash
Car 0 arrives at the carwas at 0.00.
Car 1 arrives at the carwas at 0.00.
Car 2 arrives at the carwas at 0.00.
Car 3 arrives at the carwas at 0.00.
Car 0 enters the carwash at 0.00.
Car 1 enters the carwash at 0.00.
Carwash removed 68% of Car 0's dirt.
Carwash removed 93% of Car 1's dirt.
Car 0 leaves the carwash at 2.00.
Car 1 leaves the carwash at 2.00.
Car 2 enters the carwash at 2.00.
Car 3 enters the carwash at 2.00.
Car 4 arrives at the carwas at 3.00.
Carwash removed 91% of Car 2's dirt.
Carwash removed 64% of Car 3's dirt.
Car 2 leaves the carwash at 4.00.
Car 3 leaves the carwash at 4.00.
Car 4 enters the carwash at 4.00.
Car 5 arrives at the carwas at 5.00.
Car 5 enters the carwash at 5.00.
Carwash removed 64% of Car 4's dirt.
Car 4 leaves the carwash at 6.00.
Car 6 arrives at the carwas at 7.00.
Carwash removed 58% of Car 5's dirt.
Car 6 enters the carwash at 7.00.
Car 5 leaves the carwash at 7.00.
Car 7 arrives at the carwas at 9.00.
Carwash removed 84% of Car 6's d